In [12]:
### Save API Token, if needed
%set_env QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


env: QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


In [13]:
# import of required libraries and modules
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [14]:
from qiskit.circuit.library import EfficientSU2

num_qubits = 50
reps = 2
abstract_circuit = EfficientSU2(num_qubits, reps=reps, entanglement="pairwise")

In [15]:
import numpy as np

num_parameters = abstract_circuit.num_parameters
param_values = np.random.uniform(-np.pi, np.pi, size=num_parameters)

abstract_circuit.assign_parameters(param_values, inplace=True)

In [16]:
from qiskit.circuit.library import UnitaryOverlap

abstract_circuit.barrier()
abstract_circuit = UnitaryOverlap(abstract_circuit, abstract_circuit)

In [17]:
from qiskit.quantum_info import SparsePauliOp

paulis = ["".join("Z" if i == q else "I" for i in range(num_qubits)) for q in range(num_qubits)]
abstract_observables = [SparsePauliOp(pauli) for pauli in paulis]

In [18]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
backend.name

'ibm_brisbane'

In [19]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
target_circuit = pm.run(abstract_circuit)

In [20]:
layout = target_circuit.layout
target_observables = [abs_obs.apply_layout(layout=layout) for abs_obs in abstract_observables]

In [21]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator, EstimatorOptions, Batch

# Define the primitive unified bloc (PUB) for Estimator jobs
# More on PUB: https://docs.quantum.ibm.com/api/qiskit/primitives
pub = (target_circuit, target_observables)
default_shots = 10_000

# list for saving job results
primitive_results = []

In [ ]:
# Submit Exercise 1 to 3 inside a Batch execution mode
with Batch(backend=backend) as batch:
    # Excercise-1: No mitigation (worked out for you)
    ## https://docs.quantum.ibm.com/run/configure-error-mitigation#turn-off-all-error-mitigation-and-error-suppression
    options_ex1 = EstimatorOptions() # some suppression and mitigation are enabled by default
    options_ex1.optimization_level = 0
    options_ex1.resilience_level = 0
    options_ex1.default_shots = default_shots
    
    # Instantiate `Estimator` with options
    estimator = Estimator(options=options_ex1)
    # Submit job
    job_ex1 = estimator.run(pubs=[pub])

    
    # Excercise-2: Dynamical Decoupling (DD)
    from qiskit_ibm_runtime import SamplerOptions, EstimatorOptions
    options_ex2 = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex2.default_shots = default_shots
    options_ex2.dynamical_decoupling.enable = True
    options_ex2.dynamical_decoupling.sequence_type = 'XX'
    options_ex2.dynamical_decoupling.extra_slack_distribution = 'middle'
    options_ex2.dynamical_decoupling.scheduling_method = 'alap'

    
    estimator = Estimator(options=options_ex2)
    job_ex2 = estimator.run(pubs=[pub])

    
    # Excercise-3: Measurement mitigation (TREX)
    from qiskit_ibm_runtime import EstimatorOptions
    options_ex3 = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex3.default_shots = default_shots
    options_ex3.resilience.measure_mitigation = True 
    options_ex3.resilience.measure_noise_learning.num_randomizations = 32
    options_ex3.resilience.measure_noise_learning.shots_per_randomization = 'auto'
    options_ex3.twirling.enable_measure = True

    
    estimator = Estimator(options=options_ex3)
    job_ex3 = estimator.run(pubs=[pub])

# Wait for first 3 jobs to complete. Fetch results when done
primitive_results.append(job_ex1.result())
primitive_results.append(job_ex2.result())
primitive_results.append(job_ex3.result())

In [ ]:
# Submit Exercise 4a, 4b, and 5 inside another Batch execution mode
with Batch(backend=backend) as batch:
    # Excercise-4a: Zero Noise Extrapolation (extrapolator="exponential" | noise_factors=(1, 3, 5))
    from qiskit_ibm_runtime import SamplerOptions, EstimatorOptions
    options_ex4a = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex4a.default_shots = default_shots
    options_ex4a.resilience.zne_mitigation = True
    options_ex4a.resilience.zne.noise_factors = (1,3,5)
    options_ex4a.resilience.zne.extrapolator = ('exponential')

    estimator = Estimator(options=options_ex4a)
    job_ex4a = estimator.run(pubs=[pub])

    
    # Excercise-4b: Zero Noise Extrapolation (use: extrapolator="linear" and noise_factors=(1, 3, 5))
    options_ex4b = EstimatorOptions()

    options_ex4b = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex4b.default_shots = default_shots
    options_ex4b.resilience.zne_mitigation = True
    options_ex4b.resilience.zne.noise_factors = (1,3,5)
    options_ex4b.resilience.zne.extrapolator = ('linear')
    
    estimator = Estimator(options=options_ex4b)
    job_ex4b = estimator.run(pubs=[pub])

    
    # Excercise-5: Gate Twirling + Zero Noise Extrapolation (use: extrapolator=("exponential", "linear") and noise_factors=(1, 3, 5))

    options_ex5 = EstimatorOptions(optimization_level=0, resilience_level=0)
    
    options_ex5.default_shots = default_shots
    
    options_ex5.twirling.enable_gates = True
    options_ex5.twirling.enable_measure = False
    options_ex5.twirling.num_randomizations = 'auto'
    options_ex5.twirling.shots_per_randomization = 'auto'
    options_ex5.twirling.strategy = 'active-accum'
    
    
    options_ex5.resilience.zne_mitigation = True
    options_ex5.resilience.zne.noise_factors = (1,3,5)
    options_ex5.resilience.zne.extrapolator = ('exponential' ,'linear')
    
    
    estimator = Estimator(options=options_ex5)
    job_ex5 = estimator.run(pubs=[pub])

# Wait for next 3 jobs to complete. Fetch results when done
primitive_results.append(job_ex4a.result())
primitive_results.append(job_ex4b.result())
primitive_results.append(job_ex5.result())

In [22]:
# Submit Exercise 6 in Job execution mode as it is a single job
# Excercise-6: All
from qiskit_ibm_runtime import QiskitRuntimeService

options_ex6 = EstimatorOptions(optimization_level=0, resilience_level=0)

options_ex6.default_shots = default_shots

options_ex6.dynamical_decoupling.enable = True
options_ex6.dynamical_decoupling.sequence_type = 'XX'
options_ex6.dynamical_decoupling.extra_slack_distribution = 'middle'
options_ex6.dynamical_decoupling.scheduling_method = 'alap'

options_ex6.twirling.enable_gates = True
options_ex6.twirling.enable_measure = True
options_ex6.twirling.num_randomizations = 'auto'
options_ex6.twirling.shots_per_randomization = 'auto'
options_ex6.twirling.strategy = 'active-accum'

options_ex6.resilience.measure_mitigation = True 
options_ex6.resilience.measure_noise_learning.num_randomizations = 32
options_ex6.resilience.measure_noise_learning.shots_per_randomization = 'auto'

options_ex6.resilience.zne_mitigation = True
options_ex6.resilience.zne.noise_factors = (1,3,5)
options_ex6.resilience.zne.extrapolator = ('exponential' ,'linear')    


# Note: explicitly set `mode=backend` in Job execution mode.
# Inside Batch context manager, `Estimator` knows the context/backend implicitly
# However, without the context manager, we must set `mode` explicitly
estimator = Estimator(mode=backend, options=options_ex6)
job_ex6 = estimator.run(pubs=[pub])
primitive_results.append(job_ex6.result())

In [ ]:
primitive_results